In [16]:
import pandas as pd
import numpy as np
import requests
import re
import os
import json
from pprint import pprint
from bs4 import BeautifulSoup

In [2]:
all_mineral_dict = {}
min_dict = {}
skipped = []
collapse_finder = re.compile(r"collapse[0-9]+")
FILE_NUM = 30

In [3]:
def add_key_to_dict(key, value):
    global min_dict
    ind = key.rfind(':')
    if ind == -1: return
    key = key[:ind]
    
    if key not in min_dict:
        min_dict[key] = value
    elif isinstance(min_dict[key], list):
        min_dict[key].append(value)
    else:
        min_dict[key] = [min_dict[key], value]

In [4]:
def get_intro_data(div):
    try:
        intro = div.find("div", {"id": "introdata"})
        for x in intro.find_all("div", recursive=False):
            try:
                key = x.span.text
                value = x.div.text
                add_key_to_dict(key, value)
            except Exception as e:
                print(e)
    except Exception as e:
        print(e)

In [5]:
def get_other_attributes(divs):
    for div in divs:
        try:
            rows = div.find_all('div', {'class': 'mindatarow'})
            for row in rows:
                text = row.find_all('div', recursive=False)
                if len(text) != 2: continue
                try:
                    key = text[0].text
                    if (lis := text[1].find_all('li', recursive=False)) != []:
                        value = '\n'.join([x.text for x in lis])
                    else:
                        value = text[1].text
                    add_key_to_dict(key, value)
                except Exception as e:
                    print(e)
        except Exception as e:
            print(e)
        

In [6]:
def dump_data():
    with open(f'../data/mindat/collected_data-{FILE_NUM}.json', 'w') as f:
        json.dump(all_mineral_dict, f)
    with open(f'../data/mindat/skipped_data-{FILE_NUM}.txt', 'w') as f:   
        f.write('\n'.join(str(x) for x in skipped))

In [7]:
def start_parsing(index, html_text):
    global min_dict
    global skipped
    min_dict = {}

    soup  = BeautifulSoup(html_text, "html.parser")
    try:
        container = soup.body.find("div", {"id": "mainwrap"}).find("div", {"class": "centerer"}).find("div", {"class": "mindatadiv"}).find("div", {"class": "fpbox990nl"})
        mineral_name = container.find("h1", {"class":"mineralheading"}).text
        print(mineral_name)
        divs = container.find_all("div", {"id" : collapse_finder})
        get_intro_data(divs[0])
        get_other_attributes(divs)
        
        all_mineral_dict[mineral_name] = {}
        for k, val in min_dict.items():
            if isinstance(val, list):
                val = list(set(val))
                if len(val) == 1:
                    val = val[0]
            all_mineral_dict[mineral_name][k] = val
        
    except Exception as e:
        print(e)
        print("Extraction failed")
        skipped.append(num)
    dump_data()

In [8]:
# base_url = "https://www.mindat.org"

# with open('./mindat/skipped_data-21.txt') as f:
#     nums = [int(x.strip()) for x in f]

# for num in nums:
#     url = f"{base_url}/min-{num}.html"
#     html_text = requests.get(url).text
#     start_parsing(num, html_text)
#     print(f"Page number {num} parsed.\n")


In [9]:
base_url = "https://www.mindat.org"

for num in range(4000, 5000):
    url = f"{base_url}/min-{num}.html"
    html_text = requests.get(url).text
    start_parsing(num, html_text)
    print(f"Page number {num} parsed.\n")
